In [1]:
import sys
sys.path.insert(0,'..')
from featurizer import *
import pandas as pd

### Deduplicate

In [ ]:
featurizer = Featurizer('../synthetic_outputs',
                        cdr_fname='../synthetic_data/cdr.csv', 
                        antennas_fname = '../synthetic_data/antennas.csv',
                        topups_fname='../synthetic_data/topups.csv',
                        mobiledata_fname='../synthetic_data/mobiledata.csv',
                        mobilemoney_fname='../synthetic_data/mobilemoney.csv')

cdr = pd.read_csv('../synthetic_data/cdr.csv')
topups = pd.read_csv('../synthetic_data/topups.csv')

assert featurizer.cdr.count() == len(cdr)

featurizer.cdr = featurizer.cdr.union(featurizer.cdr)
featurizer.topups = featurizer.topups.union(featurizer.topups)
assert featurizer.cdr.count() == 2*len(cdr)
assert featurizer.topups.count() == 2*len(topups)

featurizer.deduplicate()
assert featurizer.cdr.count() == len(cdr)
assert featurizer.topups.count() == len(topups)

### Filter Dates

In [ ]:
featurizer = Featurizer('../synthetic_outputs',
                        cdr_fname='../synthetic_data/cdr.csv', 
                        topups_fname='../synthetic_data/topups.csv',
                        mobiledata_fname='../synthetic_data/mobiledata.csv',
                        mobilemoney_fname='../synthetic_data/mobilemoney.csv')

for df in [featurizer.cdr, featurizer.topups, featurizer.mobiledata, featurizer.mobilemoney]:
    assert df.agg({'day':'min'}).collect()[0][0] == pd.to_datetime('2020-01-01')
    assert df.agg({'day':'max'}).collect()[0][0] == pd.to_datetime('2020-02-29')

featurizer.filter_dates('2020-01-05', '2020-02-01')
for df in [featurizer.cdr, featurizer.topups, featurizer.mobiledata, featurizer.mobilemoney]:
    assert df.agg({'day':'min'}).collect()[0][0] == pd.to_datetime('2020-01-05')
    assert df.agg({'day':'max'}).collect()[0][0] == pd.to_datetime('2020-02-01')

### Remove Spammers

In [ ]:
# Test for number of spammers identified

for spammer_threshold in [0, 1, 2]:
    
    featurizer = Featurizer('../synthetic_outputs',
                            cdr_fname='../synthetic_data/cdr.csv', 
                            topups_fname='../synthetic_data/topups.csv',
                            mobiledata_fname='../synthetic_data/mobiledata.csv',
                            mobilemoney_fname='../synthetic_data/mobilemoney.csv')

    spammers = len(featurizer.remove_spammers(spammer_threshold=spammer_threshold))
    
    cdr = pd.read_csv('../synthetic_data/cdr.csv')
    cdr['timestamp'] = pd.to_datetime(cdr['timestamp'])
    grouped = cdr.groupby(['caller_id', 'txn_type'], as_index=False).agg('count')
    num_days = (cdr['timestamp'].max() - cdr['timestamp'].min()).days + 1
    assert len(grouped[grouped['recipient_id'] > spammer_threshold*num_days]['caller_id'].unique()) == spammers

In [ ]:
# Test for removal of spammers

spammer_threshold = 1

featurizer = Featurizer('../synthetic_outputs',
                        cdr_fname='../synthetic_data/cdr.csv', 
                        topups_fname='../synthetic_data/topups.csv',
                        mobiledata_fname='../synthetic_data/mobiledata.csv',
                        mobilemoney_fname='../synthetic_data/mobilemoney.csv')

spammers = featurizer.remove_spammers(spammer_threshold=spammer_threshold)

for df in [featurizer.cdr, featurizer.topups, featurizer.mobiledata, featurizer.mobilemoney]:
    
    caller_ids = [item[0] for item in featurizer.cdr.select('caller_id').collect()]
    assert set(spammers).intersection(set(caller_ids)) == set()
    
    if 'recipient_id' in df.columns:
        recipient_ids = [item[0] for item in featurizer.cdr.select('caller_id').collect()]
        assert set(spammers).intersection(set(recipient_ids)) == set()

### Filter Outlier Days

In [ ]:
# Test for number of days identified

for num_sds in [1, 1.5, 2]:
    featurizer = Featurizer('../synthetic_outputs',
                            cdr_fname='../synthetic_data/cdr.csv', 
                            topups_fname='../synthetic_data/topups.csv',
                            mobiledata_fname='../synthetic_data/mobiledata.csv',
                            mobilemoney_fname='../synthetic_data/mobilemoney.csv')

    outliers = [pd.to_datetime(item).tz_localize(None) for item in featurizer.filter_outlier_days(num_sds)]

    cdr = pd.read_csv('../synthetic_data/cdr.csv')
    cdr['timestamp'] = pd.to_datetime(cdr['timestamp'])
    cdr['day'] = cdr['timestamp'].dt.floor('d')
    grouped = cdr.groupby('day', as_index=False).agg('count')
    u = grouped['txn_type'].mean() + num_sds*grouped['txn_type'].std()
    l = grouped['txn_type'].mean() - num_sds*grouped['txn_type'].std()
    outliers_manual = grouped[(grouped['txn_type'] < l) | (grouped['txn_type'] > u)]['day'].astype('object')
    assert list(outliers_manual.unique()) == outliers

In [ ]:
# Test for removal of days

num_sds = 2

featurizer = Featurizer('../synthetic_outputs',
                        cdr_fname='../synthetic_data/cdr.csv', 
                        topups_fname='../synthetic_data/topups.csv',
                        mobiledata_fname='../synthetic_data/mobiledata.csv',
                        mobilemoney_fname='../synthetic_data/mobilemoney.csv')

original_counts = []
for df in [featurizer.cdr, featurizer.topups, featurizer.mobiledata, featurizer.mobilemoney]:
    original_counts.append(df.select('day').distinct().count())
    
num_outliers = len(featurizer.filter_outlier_days(num_sds))

for d, df in enumerate([featurizer.cdr, featurizer.topups, featurizer.mobiledata, featurizer.mobilemoney]):
    assert(df.select('day').distinct().count() == original_counts[d] - num_outliers)

### Diagnostic Statistics

In [ ]:
featurizer = Featurizer('../synthetic_outputs',
                        cdr_fname='../synthetic_data/cdr.csv', 
                        topups_fname='../synthetic_data/topups.csv',
                        mobiledata_fname='../synthetic_data/mobiledata.csv',
                        mobilemoney_fname='../synthetic_data/mobilemoney.csv')

statistics = featurizer.diagnostic_statistics('test_output')

for fname, name in [('cdr', 'CDR')]:
    
    df = pd.read_csv('../synthetic_data/' + fname + '.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    assert((df['timestamp'].max() - df['timestamp'].min()).days + 1 == statistics[name]['Days'])
    assert(len(df) == statistics[name]['Transactions'])
    assert(len(df['caller_id'].unique()) == statistics[name]['Subscribers'])
    if 'recipient_id' in df.columns:
        assert(len(df['recipient_id'].unique()) == statistics[name]['Recipients'])

In [ ]:
featurizer.cdr.show(5)

### Diagnostic Plots

In [ ]:
featurizer = Featurizer('../synthetic_outputs',
                        cdr_fname='../synthetic_data/cdr.csv', 
                        topups_fname='../synthetic_data/topups.csv',
                        mobiledata_fname='../synthetic_data/mobiledata.csv',
                        mobilemoney_fname='../synthetic_data/mobilemoney.csv')

featurizer.diagnostic_plots('test_output')

### CDR Features

In [ ]:
import sys
sys.path.insert(0,'..')
from featurizer import *
import pandas as pd

featurizer = Featurizer('../synthetic_outputs',
                        cdr_fname='../synthetic_data/cdr.csv', 
                        antennas_fname = '../synthetic_data/antennas.csv',
                        topups_fname='../synthetic_data/topups.csv',
                        mobiledata_fname='../synthetic_data/mobiledata.csv',
                        mobilemoney_fname='../synthetic_data/mobilemoney.csv',
                        shapefiles={'regions':'../synthetic_data/regions.geojson', 
                                    'prefectures':'../synthetic_data/prefectures.geojson'})

featurizer.cdr_features(bc_chunksize=100, bc_processes=20)
featurizer.international_features()
featurizer.location_features()
featurizer.mobiledata_features()

In [1]:
import sys
sys.path.insert(0,'..')
from featurizer import *
import pandas as pd

featurizer = Featurizer('../synthetic_outputs',
                        cdr_fname='../synthetic_data/cdr.csv', 
                        antennas_fname = '../synthetic_data/antennas.csv',
                        recharges_fname='../synthetic_data/recharges.csv',
                        mobiledata_fname='../synthetic_data/mobiledata.csv',
                        mobilemoney_fname='../synthetic_data/mobilemoney.csv',
                        shapefiles={'regions':'../synthetic_data/regions.geojson', 
                                    'prefectures':'../synthetic_data/prefectures.geojson'})

featurizer.cdr_features()
featurizer.international_features()
featurizer.location_features()
featurizer.mobiledata_features()
featurizer.mobilemoney_features()
featurizer.recharges_features()

Loading CDR...
Loading antennas...
Loading recharges...
Loading mobile data...
Loading mobile money...
Calculating CDR features...
Calculating international features...
Calculating spatial features...


/home/em/.conda/envs/py3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326'})
  "(%s != %s)" % (left_df.crs, right_df.crs)
/home/em/.conda/envs/py3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326'})
  "(%s != %s)" % (left_df.crs, right_df.crs)


Calculating mobile data features...
Calculating mobile money features...
Calculating recharges features...


In [2]:
featurizer.all_features()

AnalysisException: USING column `name` cannot be resolved on the right side of the join. The right-side columns: [mobiledata_caller_id, mobiledata_total_volume, mobiledata_mean_volume, mobiledata_min_volume, mobiledata_max_volume, mobiledata_std_volume, mobiledata_num_days, mobiledata_num_transactions];